In [ ]:
%load_ext autoreload
%autoreload 2

In [11]:
from model.depth import convert_video_to_depth
from model.dmd import (
    ROOT,
    convert_depth_images_to_video,
    convert_frames_to_video,
    extract_frames,
)

In [12]:
session = 'gA_1_s1_2019-03-08T09;31;15+01;00'
input_video_path = ROOT / session / f'{session}_rgb_body.mp4'
annotations_file_path = ROOT / session / f'{session}.json'
source_type = 'rgb'

In [ ]:
# Extract frames from source video based on annotations
extract_frames(
    input_video_path=input_video_path,
    annotations_file_path=annotations_file_path,
    force_overwrite=False,
    source_type=source_type,
)

In [ ]:
# RGB frames to video (also resizes to 518x518)
convert_frames_to_video(session, source_type=source_type, preset='slow', crf=10)

In [ ]:
# Video Depth Anything
convert_video_to_depth(session, source_type=source_type)

In [5]:
# Visualization with a color map
convert_depth_images_to_video(session, 'anomal', 12, extension='png', fps=30)

In [ ]:
# import shutil

# all_dirs = sorted(
#     [p for p in (ROOT / session).rglob('video_depth_anything') if p.is_dir()]
# )
# for dir in all_dirs:
#     shutil.rmtree(dir)
#     print(f'Removed {dir}')